In [1]:
# #!/usr/bin/env python3
# import os
# import logging
# from pathlib import Path
# import requests from optkeras.optkeras import OptKeras
# import optkeras
# import pickle
# from os import listdir
# from numpy import asarray
# from numpy import save
# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
# import numpy as np
# from keras import layers
# from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
# from keras.models import Model, load_model
# from keras.utils import layer_utils
# from keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint
# from keras.callbacks import EarlyStopping
# import keras.backend as K
# from keras.applications.vgg16 import VGG16
# from keras.models import Model,load_model
# from keras.optimizers import RMSprop

# import optuna
# from zipfile import ZipFile

# logging.basicConfig(level=logging.DEBUG)

# #Import Pegasus API
# from Pegasus.api import *

# #Downloading dataset
# zipurl = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip'    
# r = requests.get(zipurl, stream = True) 
  
# with open("dataset.zip","wb") as pdf: 
#     for chunk in r.iter_content(chunk_size=1024): 
#          if chunk: 
#             pdf.write(chunk)
            
# # Extract all the contents of zip file in current directory            
# with ZipFile('dataset.zip', 'r') as zipObj:
#     zipObj.extractall()

In [2]:
from optkeras.optkeras import OptKeras
import optkeras
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna

Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [3]:
# !sudo pip3 install --upgrade pip

In [4]:
# !sudo pip3 install --upgrade pip keras==2.1.5 tensorflow==1.13.1 numpy pandas pillow sklearn optuna scikit-image  optkeras

In [5]:

import os
import signal
import sys
import time
import traceback

from pathlib import Path

In [8]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from glob import glob
from zipfile import ZipFile

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *

#Properties
props = Properties()
props["pegasus.data.configuration"] = "condorio"
props["dagman.retry"] = "100"
props.write()

#Replica Catalog
rc = ReplicaCatalog()
input_files = glob('*.jpg')
input_files.sort()
in_files=[]

import os
import pickle
        
checkpoint_file = "checkpoint_vgg.pkl"
if not os.path.isfile(checkpoint_file):
    with open(checkpoint_file,'wb') as file:
        pickle.dump("0", file)


for file in input_files:
    in_files.append(File(file))
    rc.add_replica("local", File(file), str(Path(".").resolve() / file))  
rc.add_replica("local", File(checkpoint_file), str(Path(".").resolve() / file)) 
rc.write()


#Transformation
pre_process_resize = Transformation( "preprocess1.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/preprocess1.py",
        is_stageable=True)

pre_process_augment = Transformation( "Augmentation.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Augmentation.py",
        is_stageable=True)

data_split  = Transformation( "Data_Split.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Data_Split.py",
        is_stageable=True)

vgg_model  = Transformation( "VGG_model.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/VGG_model.py",
        is_stageable=True)

hpo =  Transformation( "hpo_checkpointing.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/hpo_checkpointing.py",
        is_stageable=True)
                    
tc = TransformationCatalog()\
    .add_transformations(pre_process_resize,pre_process_augment,data_split,vgg_model,hpo)\
    .write()

#Workflow
wf = Workflow("Cats_and_Dogs", infer_dependencies=True)


resized_images = File('resized_images.txt')
all_files = [File("resized_{}".format(f.lfn)) for f in in_files]
labels = File('labels.txt')

job_preprocess1 = Job(pre_process_resize)\
                    .add_inputs(*in_files)\
                    .add_outputs(*all_files,resized_images,labels) 

aug_images_txt = File('augmentation.txt')
aug_labels_txt = File('aug_labels.txt')
augmented_files = []
for f in all_files:
    augmented_files.extend([File(str(f).replace("{}".format(os.path.splitext(str(f))[0]), "Aug_{}_{}".format(os.path.splitext(str(f))[0],i))) for i in range(3)])

    
job_preprocess2 = Job(pre_process_augment)\
                    .add_inputs(*all_files,labels)\
                    .add_outputs(aug_images_txt,aug_labels_txt,*augmented_files)

training_data = File('training.pkl')
testing_data = File('testing.pkl')
val_data = File('validation.pkl')

job_data_split = Job(data_split)\
                    .add_inputs(*augmented_files,labels)\
                    .add_outputs(training_data,testing_data,val_data)

model = File('model.h5')

job_vgg_model = Job(vgg_model)\
                    .add_inputs(*augmented_files,training_data,testing_data,val_data)\
                    .add_outputs(model)

model1 = File('model_hpo.h5')
optuna_csv = File("Optuna_log_file.csv")


job_hpo = Job(hpo)\
                    .add_checkpoint(File(checkpoint_file), stage_out=True)\
                    .add_inputs(model,*augmented_files,training_data,testing_data,val_data)\
                    .add_outputs(model1)\
                    .add_profiles(Namespace.PEGASUS, key="maxwalltime", value=5)


wf.add_jobs(job_preprocess1,job_preprocess2,job_data_split,job_vgg_model,job_hpo)                                    

In [ ]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.09.10 20:33:15.923 UTC:
2020.09.10 20:33:15.928 UTC:   -----------------------------------------------------------------------
2020.09.10 20:33:15.937 UTC:   File for submitting this DAG to HTCondor           : Cats_and_Dogs-0.dag.condor.sub
2020.09.10 20:33:15.942 UTC:   Log of DAGMan debugging messages                 : Cats_and_Dogs-0.dag.dagman.out
2020.09.10 20:33:15.960 UTC:   Log of 

[--------------------------------------------------]   0.0% ..--------------------------------------------------]   0.0% ..Running (Completed: 0, Queued: 0, Running: 0, Failed: 0)--------------------------------------------------]   0.0% ..Running (Completed: 0, Queued: 0, Running: 1, Failed: 0)--------------------------------------------------]   0.0% ..Running (Completed: 0, Queued: 0, Running: 0, Failed: 0)##------------------------------------------------]   3.3% ..Running (Completed: 1, Queued: 7, Running: 0, Failed: 0)##------------------------------------------------]   3.3% ..Running (Completed: 1, Queued: 0, Running: 1, Failed: 0)############--------------------------------------]  23.3% ..Running (Completed: 7, Queued: 0, Running: 1, Failed: 0)############--------------------------------------]  23.3% ..Running (Completed: 7, Queued: 0, Running: 1, Failed: 0)############--------------------------------------]  23.3% ..Running (Completed: 7, Queued: 0, Running: 1, Failed: 0)##

[-------------------]  60.0% ..Running (Completed: 18, Queued: 2, Running: 1, Failed: 0)##############################--------------------]  60.0% ..Running (Completed: 18, Queued: 0, Running: 3, Failed: 0)##############################--------------------]  60.0% ..Running (Completed: 18, Queued: 0, Running: 2, Failed: 0)################################------------------]  63.3% ..Running (Completed: 19, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Q

[ Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#####################

[----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queued: 0, Running: 1, Failed: 0)#################################-----------------]  66.7% ..Running (Completed: 20, Queu

[ueued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)#######################

[---------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, 

[ Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)################################

[---------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, 

[ Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)########################################----------]  80.0% ..Running (Completed: 24, Queued: 0, Running: 1, Failed: 0)################################

In [ ]:
# augmented_files = []
# for f in all_files:
#     augmented_files.extend([File(str(f).replace("{}".format(os.path.splitext(str(f))[0]), "Aug_{}_{}".format(os.path.splitext(str(f))[0],i))) for i in range(3)])

In [ ]:
# len(augmented_files)

In [ ]:
# len(all_files)